In [1]:
from esda.moran import Moran
from libpysal.weights import Queen, KNN
from libpysal.cg import get_points_dist
from shapely.geometry import Polygon
from pysal.lib import cg as geometry
from pysal.lib import weights


import pandas as pd
import geopandas as gpd
import numpy as np
import os
import seaborn as sns
import pandas as pd
import pysal as ps
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import cluster
from matplotlib.colors import ListedColormap
import networkx as nx

import statsmodels.api as sm
from scipy.stats import linregress, pearsonr, spearmanr
import statsmodels.formula.api as smf


In [2]:
CountBySpecialty = pd.read_csv('./CountBySpecialty.csv')
CountBySpecialty['PerCapita'] = CountBySpecialty['Count']/CountBySpecialty['msa_pop']
df_cbsa = CountBySpecialty[['cbsa','msa_name','msa_pop']].drop_duplicates()
df_cbsa.columns = ['cbsacode','msa_name','Population']
cbsacode = df_cbsa.cbsacode
msa_name = df_cbsa.msa_name
Population = df_cbsa.Population
Specialty = CountBySpecialty.Specialty.unique()
new_df = pd.DataFrame({
    'cbsacode':cbsacode,
    'msa_name':msa_name,
    'Population':Population,

    # 'Specialty':[],
    # 'Specialty_Count':[],
})
for s in Specialty:
    new_df[s] = np.nan

for c in cbsacode:
    for s in Specialty:
        try:
            sc = CountBySpecialty[(CountBySpecialty['cbsa']==c)&(CountBySpecialty['Specialty']==s)]['Count'].iloc[0]
            new_df.loc[new_df['cbsacode'] == c, s] = sc
        except:
            new_df.loc[new_df['cbsacode'] == c, s] = 0
            
new_df['TOTAL_PROVIDER'] = new_df.iloc[:,3:].sum(axis=1)
new_df['PROVIDER_PERCAPITA'] = new_df['TOTAL_PROVIDER']/new_df['Population']
CountBySpecialty.head()

,Unnamed: 0,cbsa,msa_name,msa_pop,LSAD,Specialty,Count,PerCapita
0,0,10100.0,"Aberdeen, SD",41790,Micropolitan Statistical Area,PATHOLOGY,2,0.000048
1,1,10140.0,"Aberdeen, WA",77290,Micropolitan Statistical Area,PATHOLOGY,2,0.000026
2,2,10180.0,"Abilene, TX",181591,Metropolitan Statistical Area,PATHOLOGY,4,0.000022
3,3,10220.0,"Ada, OK",38396,Micropolitan Statistical Area,PATHOLOGY,2,0.000052
4,4,10300.0,"Adrian, MI",97520,Micropolitan Statistical Area,PATHOLOGY,11,0.000113


In [5]:
county_population = pd.read_csv("./ACSST5Y2020.S0102_2025-06-27T165935/ACSST5Y2020.S0102-Data.csv")
county_population = county_population.iloc[1:, :3]
county_population['FIPS'] = [int(f[-5:]) for f in county_population.GEO_ID]
county_population = county_population.fillna(0)
county_population['Population'] = county_population['S0102_C01_001E'].astype(int)

county_population

/var/folders/5r/5gd5nnz10gb73gwmbv5yxjdc0000gp/T/ipykernel_53006/1560652256.py:1: DtypeWarning: Columns (2,4,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,56,57,58,59,60,61,62,63,64,65,66,68,69,70,71,72,73,74,75,76,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258

,GEO_ID,NAME,S0102_C01_001E,FIPS,Population
1,0500000US01001,"Autauga County, Alabama",55639,1001,55639
2,0500000US01003,"Baldwin County, Alabama",218289,1003,218289
3,0500000US01005,"Barbour County, Alabama",0,1005,0
4,0500000US01007,"Bibb County, Alabama",0,1007,0
5,0500000US01009,"Blount County, Alabama",57755,1009,57755
...,...,...,...,...,...
3217,0500000US72145,"Vega Baja Municipio, Puerto Rico",0,72145,0
3218,0500000US72147,"Vieques Municipio, Puerto Rico",0,72147,0
3219,0500000US72149,"Villalba Municipio, Puerto Rico",0,72149,0
3220,0500000US72151,"Yabucoa Municipio, Puerto Rico",0,72151,0


In [6]:
# TOC_2010_2014 = pd.read_csv('/Users/tg2426/Documents/Python/MedicalEconomy/DATA/IHME_USA_HEALTH_CARE_SPENDING_CAUSE_COUNTY_2010_2019_TABLE_2_COUNTY_TOC_PAYER_2010_2014_Y2025M02D13.CSV')
TOC_2015_2019 = pd.read_csv('./IHME_USA_HEALTH_CARE_SPENDING_CAUSE_COUNTY_2010_2019_TABLE_3_COUNTY_TOC_PAYER_2015_2019_Y2025M02D13.CSV')

TOC_2019 = TOC_2015_2019[(TOC_2015_2019['year_id']==2019)&(TOC_2015_2019['payer'].isin(['mdcd','mdcr','oop','priv']))&(TOC_2015_2019['age_name'].isin(['All ages']))]
TOC_2019 = TOC_2019.merge(county_population[['FIPS','Population']], left_on='fips', right_on='FIPS') 
TOC_2019['vol_per_capita_mean'] = TOC_2019['vol_per_capita_mean']/(1000)
TOC_2019['vol_mean'] = TOC_2019['vol_per_capita_mean']*TOC_2019['Population']
TOC_2019 = TOC_2019.groupby(['year_id','location_name','fips','acause']).agg({
    'Population':sum,
    'spend_mean': np.mean,
    'spend_per_capita_mean':np.mean,
    'vol_mean':np.mean,
    'vol_per_capita_mean':np.mean,

}).reset_index()
TOC_2019

,year_id,location_name,fips,acause,Population,spend_mean,spend_per_capita_mean,vol_mean,vol_per_capita_mean
0,2019,Abbeville County,45001,all,688296,5.702034e+06,232.675000,33010.579588,1.342876
1,2019,Acadia Parish,22001,all,1746388,1.484721e+07,256.148214,120249.015914,1.927964
2,2019,Accomack County,51001,all,911680,7.736954e+06,231.714643,45764.835914,1.405554
3,2019,Ada County,16001,all,13145244,1.211525e+08,248.030357,552091.529216,1.175981
4,2019,Adair County,19001,all,0,2.118709e+06,281.832143,0.000000,1.346633
...,...,...,...,...,...,...,...,...,...
3138,2019,Yuma County,4027,all,5934068,3.939104e+07,179.212143,207629.557596,0.979704
3139,2019,Yuma County,8125,all,0,2.176632e+06,217.623571,0.000000,1.162793
3140,2019,Zapata County,48505,all,0,2.146772e+06,153.473214,0.000000,1.089995
3141,2019,Zavala County,48507,all,0,1.774011e+06,179.783929,0.000000,1.271200


In [9]:
# county_shape = gpd.read_file('./DATA/cb_2018_us_county_500k/cb_2018_us_county_500k.shp')
# county_shape['FIPS'] = county_shape.GEOID.astype('int')
cbsa_shape = gpd.read_file('./tl_2020_us_cbsa/tl_2020_us_cbsa.shp')
cbsa_shape['cbsacode'] = cbsa_shape['GEOID'].astype('int')
# state_shape = gpd.read_file('./DATA/tl_2020_us_state/tl_2020_us_state.shp')
# # state_shape['STUSPS'] = [s[-2:] for s in state_shape['NAME']]
# # state_shape['STUSPS'] = state_shape['STATEFP']
# state_shape.head()

In [10]:
cbsa2county = pd.read_csv('./cbsa2county.csv')
TOC_2019['FIPS'] = TOC_2019.fips
# # TOT_data = TOT_data[TOT_data['year_id']==2019]
TOT_data = TOC_2019.merge(cbsa2county[['cbsacode','FIPS','cbsatitle',]], on='FIPS',how='right')
cbsa_TotalSpend = TOT_data.groupby(['cbsacode','cbsatitle','year_id',]).agg({
    'spend_mean':sum,
    # 'spend_lower':sum,
    # 'spend_upper':sum,
    'spend_per_capita_mean':sum,
    # 'spend_per_capita_lower':sum,
    # 'spend_per_capita_upper':sum,
    # 'vol_per_capita_mean':sum,
    # 'spend_per_capita_lower':sum,
    # 'spend_per_capita_upper':sum,
    'vol_mean': sum,
    'vol_per_capita_mean':sum,
    
})
cbsa_TotalSpend.reset_index(inplace=True)
cbsa_TotalSpend['STUSPS'] = [s[-2:] for s in cbsa_TotalSpend['cbsatitle']]
cbsa_TotalSpend = gpd.GeoDataFrame(cbsa_TotalSpend.merge(cbsa_shape, on='cbsacode',))
cbsa_TotalSpend

,cbsacode,cbsatitle,year_id,spend_mean,spend_per_capita_mean,vol_mean,vol_per_capita_mean,STUSPS,CSAFP,CBSAFP,...,NAME,NAMELSAD,LSAD,MEMI,MTFCC,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,10100,"Aberdeen, SD",2019.0,1.096086e+07,540.707857,66218.890114,3.483084,SD,NaN,10100,...,"Aberdeen, SD","Aberdeen, SD Micro Area",M2,2,G3110,7353004128,111384253,+45.5194353,-098.7007003,"POLYGON ((-98.72248 45.24376, -98.72542 45.243..."
1,10140,"Aberdeen, WA",2019.0,1.701542e+07,225.572500,85798.326408,1.163068,WA,NaN,10140,...,"Aberdeen, WA","Aberdeen, WA Micro Area",M2,2,G3110,4924794342,834310602,+47.1137324,-123.8267349,"POLYGON ((-124.30542 47.24464, -124.30637 47.2..."
2,10180,"Abilene, TX",2019.0,3.453346e+07,620.445357,163930.238152,3.940629,TX,NaN,10180,...,"Abilene, TX","Abilene, TX Metro Area",M1,1,G3110,7105668292,36756436,+32.4520222,-099.7187428,"POLYGON ((-100.14654 32.52279, -100.14642 32.5..."
3,10220,"Ada, OK",2019.0,8.390485e+06,220.290357,48926.069357,1.274614,OK,NaN,10220,...,"Ada, OK","Ada, OK Micro Area",M2,2,G3110,1865856327,12574651,+34.7214152,-096.6917804,"POLYGON ((-96.93251 34.68042, -96.93252 34.680..."
4,10300,"Adrian, MI",2019.0,2.825124e+07,283.534643,148101.346586,1.506473,MI,220,10300,...,"Adrian, MI","Adrian, MI Micro Area",M2,2,G3110,1941526154,30497573,+41.8960225,-084.0743563,"POLYGON ((-84.36198 41.89876, -84.36198 41.898..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,49660,"Youngstown-Warren, OH",2019.0,1.190462e+08,553.321786,660235.112266,3.097936,OH,566,49660,...,"Youngstown-Warren-Boardman, OH-PA","Youngstown-Warren-Boardman, OH-PA Metro Area",M1,1,G3110,4408441873,109322747,+41.2364600,-080.5617836,"POLYGON ((-81.00229 41.13419, -81.00232 41.145..."
864,49700,"Yuba City, CA",2019.0,4.001998e+07,444.988571,226161.670669,2.590823,CA,472,49700,...,"Yuba City, CA","Yuba City, CA Metro Area",M1,1,G3110,3197863368,45895634,+39.1609692,-121.5118327,"POLYGON ((-121.62376 39.29562, -121.62339 39.2..."
865,49740,"Yuma, AZ",2019.0,3.939104e+07,179.212143,207629.557596,0.979704,AZ,NaN,49740,...,"Yuma, AZ","Yuma, AZ Metro Area",M1,1,G3110,14280774789,13253159,+32.7739424,-113.9109050,"POLYGON ((-114.79193 32.56682, -114.79186 32.5..."
866,49780,"Zanesville, OH",2019.0,1.959898e+07,226.992500,108026.998463,1.255646,OH,198,49780,...,"Zanesville, OH","Zanesville, OH Micro Area",M2,2,G3110,1721165826,20823924,+39.9660457,-081.9435064,"POLYGON ((-82.19479 40.02069, -82.19477 40.021..."


In [17]:
county_DIABETES_TYPE1 = pd.read_csv("/Users/tg2426/Documents/Python/MedicalEconomy/DATA/AGGREGATE_TABLES/DIABETES_TYPE1_PAYER_2019.csv")
county_DIABETES_TYPE2 = pd.read_csv("/Users/tg2426/Documents/Python/MedicalEconomy/DATA/AGGREGATE_TABLES/DIABETES_TYPE2_PAYER_2019.csv")
county_DIABETES = pd.concat((county_DIABETES_TYPE1, county_DIABETES_TYPE2))
county_DIABETES['FIPS'] = county_DIABETES.fips.astype('int')
county_DIABETES['vol_mean'] = county_DIABETES['vol_mean']/10e3
county_DIABETES = county_DIABETES[county_DIABETES['payer'].isin(['mdcr','mdcd'])]
county_DIABETES = county_DIABETES.groupby(['year_id','FIPS',]).agg({
    'spend_mean': sum,
    'vol_mean': sum
}).reset_index()
county_DIABETES.columns = ['year_id', 'FIPS','diabetes_spend_mean','diabetes_vol_mean']
county_CVD = pd.read_csv("/Users/tg2426/Documents/Python/MedicalEconomy/DATA/AGGREGATE_TABLES/CVD_PAYER_2019.csv")
county_CVD['FIPS'] = county_CVD.fips.astype('int')
county_CVD['vol_mean'] = county_CVD['vol_mean']/10e3
county_CVD = county_CVD[county_CVD['payer'].isin(['mdcr','mdcd','priv','oop'])]
county_CVD = county_CVD.groupby(['year_id','FIPS',]).agg({
    'spend_mean': sum,
    'vol_mean': sum
}).reset_index()
county_CVD.columns = ['year_id', 'FIPS','cvd_spend_mean','cvd_vol_mean']

county_disease = county_DIABETES.merge(county_CVD, on=['year_id','FIPS'])
county_disease

,year_id,FIPS,diabetes_spend_mean,diabetes_vol_mean,cvd_spend_mean,cvd_vol_mean
0,2019,1001,10518799.91,4518.333,2.331248e+07,14720.034
1,2019,1003,61882912.83,5464.755,1.774186e+08,18682.932
2,2019,1005,8045658.21,6289.610,1.261832e+07,15998.742
3,2019,1007,5279817.46,5498.587,9.908353e+06,14979.639
4,2019,1009,14050389.02,5420.719,3.146765e+07,16899.396
...,...,...,...,...,...,...
3138,2019,56037,3743814.24,2881.013,1.236292e+07,11733.974
3139,2019,56039,1946073.08,2455.415,7.997628e+06,10710.314
3140,2019,56041,1926936.68,2842.930,4.785442e+06,11127.553
3141,2019,56043,1233161.26,3299.469,4.330253e+06,14101.951


In [21]:
cbsa_disease = county_disease.merge(cbsa2county[['cbsacode','FIPS','cbsatitle',]], on='FIPS',how='left')
cbsa_disease = cbsa_disease.groupby(['cbsacode','cbsatitle','year_id',]).agg({
    'diabetes_spend_mean':sum,
    'diabetes_vol_mean':sum,
    'cvd_spend_mean':sum,
    'cvd_vol_mean':sum,
}).reset_index()
# cbsa_disease.columns = ['cbsacode','cbsatitle','year_id','diabetes_spend','diabetes_encounter']
cbsa_disease

,cbsacode,cbsatitle,year_id,diabetes_spend_mean,diabetes_vol_mean,cvd_spend_mean,cvd_vol_mean
0,10100.0,"Aberdeen, SD",2019,6.519297e+06,7223.228,2.049080e+07,37904.915
1,10140.0,"Aberdeen, WA",2019,1.613358e+07,3894.963,3.822415e+07,12532.562
2,10180.0,"Abilene, TX",2019,3.118973e+07,16095.673,9.296478e+07,53801.460
3,10220.0,"Ada, OK",2019,8.184060e+06,4147.196,1.925842e+07,15510.054
4,10300.0,"Adrian, MI",2019,2.518448e+07,5121.615,7.435924e+07,15276.613
...,...,...,...,...,...,...,...
913,49660.0,"Youngstown-Warren, OH",2019,1.440199e+08,10773.954,3.687416e+08,29375.217
914,49700.0,"Yuba City, CA",2019,3.667962e+07,10171.667,9.261737e+07,33387.224
915,49740.0,"Yuma, AZ",2019,4.801002e+07,3622.252,1.104682e+08,11245.919
916,49780.0,"Zanesville, OH",2019,2.572229e+07,5704.709,5.743038e+07,15215.353


In [22]:
cbsa_TOTAL = gpd.GeoDataFrame(cbsa_TotalSpend.merge(new_df, on='cbsacode',how='inner', ))
cbsa_TOTAL

,cbsacode,cbsatitle,year_id,spend_mean,spend_per_capita_mean,vol_mean,vol_per_capita_mean,STUSPS,CSAFP,CBSAFP,...,UNDEFINED NON-PHYSICIAN TYPE (SPECIFY),PERIPHERAL VASCULAR DISEASE,ORAL AND MAXILLOFACIAL RADIOLOGY,DENTAL ANESTHESIOLOGY,PROSTHODONTICS,ORAL MEDICINE,ORAL AND MAXILLOFACIAL PATHOLOGY,OROFACIAL PAIN,TOTAL_PROVIDER,PROVIDER_PERCAPITA
0,10100,"Aberdeen, SD",2019.0,1.096086e+07,540.707857,66218.890114,3.483084,SD,NaN,10100,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,326.0,0.007801
1,10140,"Aberdeen, WA",2019.0,1.701542e+07,225.572500,85798.326408,1.163068,WA,NaN,10140,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,244.0,0.003157
2,10180,"Abilene, TX",2019.0,3.453346e+07,620.445357,163930.238152,3.940629,TX,NaN,10180,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1016.0,0.005595
3,10220,"Ada, OK",2019.0,8.390485e+06,220.290357,48926.069357,1.274614,OK,NaN,10220,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,395.0,0.010288
4,10300,"Adrian, MI",2019.0,2.825124e+07,283.534643,148101.346586,1.506473,MI,220,10300,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,387.0,0.003968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845,49660,"Youngstown-Warren, OH",2019.0,1.190462e+08,553.321786,660235.112266,3.097936,OH,566,49660,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1885.0,0.004425
846,49700,"Yuba City, CA",2019.0,4.001998e+07,444.988571,226161.670669,2.590823,CA,472,49700,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,538.0,0.002929
847,49740,"Yuma, AZ",2019.0,3.939104e+07,179.212143,207629.557596,0.979704,AZ,NaN,49740,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,711.0,0.003335
848,49780,"Zanesville, OH",2019.0,1.959898e+07,226.992500,108026.998463,1.255646,OH,198,49780,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,577.0,0.006686


In [23]:
GDP = pd.read_csv("./CAGDP1__ALL_AREAS_2001_2023.csv", encoding="latin1")

GDP_county = GDP[GDP['Unit']=='Thousands of dollars']
GDP_county['FIPS'] = [int(i[2:-1]) for i in GDP_county['GeoFIPS']]
GDP_county = GDP_county[['FIPS','GeoName','2019']]
# GDP_county['GDP_2019'] = GDP_county['2019'].astype(float)
GDP_county.columns = ['FIPS','GeoName','GDP_2019']
GDP_county

GDP_cbsa = GDP_county.merge(cbsa2county[['cbsacode','FIPS','cbsatitle','statename']], on='FIPS')
GDP_cbsa['GDP_2019'] = GDP_cbsa['GDP_2019'].astype(float)
GDP_cbsa = GDP_cbsa.groupby(['cbsacode','cbsatitle',]).agg({
    'GDP_2019':sum
}).reset_index()
GDP_cbsa

/var/folders/5r/5gd5nnz10gb73gwmbv5yxjdc0000gp/T/ipykernel_53006/1638200786.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GDP_county['FIPS'] = [int(i[2:-1]) for i in GDP_county['GeoFIPS']]


,cbsacode,cbsatitle,GDP_2019
0,10100,"Aberdeen, SD",2423280.0
1,10140,"Aberdeen, WA",2697837.0
2,10180,"Abilene, TX",7792209.0
3,10220,"Ada, OK",2100430.0
4,10300,"Adrian, MI",3051416.0
...,...,...,...
908,49660,"Youngstown-Warren, OH",17218410.0
909,49700,"Yuba City, CA",7100072.0
910,49740,"Yuma, AZ",7968248.0
911,49780,"Zanesville, OH",3846949.0


In [25]:
# cbsa_new = cbsa_TOTAL.merge(cbsa_disease, on=['cbsacode','cbsatitle','year_id',])
cbsa_new = cbsa_TOTAL.merge(GDP_cbsa, on=['cbsacode','cbsatitle'])
# cbsa_new.to_file("CBSA_ALL/CBSA_ALL.shp", driver="ESRI Shapefile")
cbsa_new

,cbsacode,cbsatitle,year_id,spend_mean,spend_per_capita_mean,vol_mean,vol_per_capita_mean,STUSPS,CSAFP,CBSAFP,...,PERIPHERAL VASCULAR DISEASE,ORAL AND MAXILLOFACIAL RADIOLOGY,DENTAL ANESTHESIOLOGY,PROSTHODONTICS,ORAL MEDICINE,ORAL AND MAXILLOFACIAL PATHOLOGY,OROFACIAL PAIN,TOTAL_PROVIDER,PROVIDER_PERCAPITA,GDP_2019
0,10100,"Aberdeen, SD",2019.0,1.096086e+07,540.707857,66218.890114,3.483084,SD,NaN,10100,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,326.0,0.007801,2423280.0
1,10140,"Aberdeen, WA",2019.0,1.701542e+07,225.572500,85798.326408,1.163068,WA,NaN,10140,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,244.0,0.003157,2697837.0
2,10180,"Abilene, TX",2019.0,3.453346e+07,620.445357,163930.238152,3.940629,TX,NaN,10180,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1016.0,0.005595,7792209.0
3,10220,"Ada, OK",2019.0,8.390485e+06,220.290357,48926.069357,1.274614,OK,NaN,10220,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,395.0,0.010288,2100430.0
4,10300,"Adrian, MI",2019.0,2.825124e+07,283.534643,148101.346586,1.506473,MI,220,10300,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,387.0,0.003968,3051416.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
840,49660,"Youngstown-Warren, OH",2019.0,1.190462e+08,553.321786,660235.112266,3.097936,OH,566,49660,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1885.0,0.004425,17218410.0
841,49700,"Yuba City, CA",2019.0,4.001998e+07,444.988571,226161.670669,2.590823,CA,472,49700,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,538.0,0.002929,7100072.0
842,49740,"Yuma, AZ",2019.0,3.939104e+07,179.212143,207629.557596,0.979704,AZ,NaN,49740,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,711.0,0.003335,7968248.0
843,49780,"Zanesville, OH",2019.0,1.959898e+07,226.992500,108026.998463,1.255646,OH,198,49780,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,577.0,0.006686,3846949.0
